In [1]:
!pip install -U qdrant_client
!pip install -U sentence_transformers
!pip install langchain
!pip install langchain-community
!pip install transformers
!pip install tensorflow
!pip install scipy scikit-learn

  Using cached protobuf-5.28.2-cp38-abi3-manylinux2014_x86_64.whl.metadata (592 bytes)
Using cached protobuf-5.28.2-cp38-abi3-manylinux2014_x86_64.whl (316 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.5
    Uninstalling protobuf-4.25.5:
      Successfully uninstalled protobuf-4.25.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires protobuf<5,>=3.20, but you have protobuf 5.28.2 which is incompatible.
google-ai-generativelanguage 0.6.6 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.28.2 which is incompatible.
google-cloud-datastore 2.19.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.28.2 which is incompatible.
google-cloud-fires

In [2]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import re
from langchain.text_splitter import RecursiveCharacterTextSplitter # Importer RecursiveCharacterTextSplitter1

In [3]:
data = pd.read_csv('/content/drive/MyDrive/IA Chrétien/backend/bible_cleaned.csv')
data.head()

,Book Name,Chapter,Verse,Text
0,Genèse,1,1,"¶ au commencement, dieu créa les cieux et la t..."
1,Genèse,1,2,la terre était informe et vide: il y avait des...
2,Genèse,1,3,¶ dieu dit: que la lumière soit! et la lumière...
3,Genèse,1,4,dieu vit que la lumière était bonne; et dieu s...
4,Genèse,1,5,"dieu appela la lumière jour, et il appela les ..."


In [4]:

# Télécharger les stopwords en français
nltk.download('stopwords')
stop_words = set(stopwords.words('french'))

 #fonction pour nettoyer les caractères spéciaux
def nettoyer_caractere(texte):
    texte = texte.lower() #conversion en minuscules
    #texte = re.sub(r'[^a-zA-Zéèêàâîôùûç\s]', '',texte) #supprimer les caractères spéciaux
    mots = texte.split() #diviser le corpus(division par espace)
      #mots = [mot for mot in mots if mot not in stop_words] #supprimer les mots courants
    return ' '.join(mots) #mette les mots en chaîne de caractères



# Initialisation du RecursiveTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024)

# Fonction pour appliquer le splitter sur chaque ligne de context
def split_text(text):
    return text_splitter.split_text(text) # Use split_text instead of split

# Application du text splitter sur chaque ligne de context
data['text_chunks'] = data['Text'].apply(lambda x: split_text(x))

# Affichage des résultats
print(data['Text'].iloc[0])
print(data['text_chunks'].iloc[0])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


¶ au commencement, dieu créa les cieux et la terre.
['¶ au commencement, dieu créa les cieux et la terre.']


In [5]:
from huggingface_hub import notebook_login
notebook_login()
# BAAI/bge-m3 should be a separate line, likely used in a model loading context.
# For example:
# from transformers import AutoModelForSeq2SeqLM
# model = AutoModelForSeq2SeqLM.from_pretrained("BAAI/bge-m3")

In [6]:
data

,Book Name,Chapter,Verse,Text,text_chunks
0,Genèse,1,1,"¶ au commencement, dieu créa les cieux et la t...","[¶ au commencement, dieu créa les cieux et la ..."
1,Genèse,1,2,la terre était informe et vide: il y avait des...,[la terre était informe et vide: il y avait de...
2,Genèse,1,3,¶ dieu dit: que la lumière soit! et la lumière...,[¶ dieu dit: que la lumière soit! et la lumièr...
3,Genèse,1,4,dieu vit que la lumière était bonne; et dieu s...,[dieu vit que la lumière était bonne; et dieu ...
4,Genèse,1,5,"dieu appela la lumière jour, et il appela les ...","[dieu appela la lumière jour, et il appela les..."
...,...,...,...,...,...
31097,Apocalypse,22,17,et l'esprit et l'épouse disent: viens. et que ...,[et l'esprit et l'épouse disent: viens. et que...
31098,Apocalypse,22,18,je le déclare à quiconque entend les paroles d...,[je le déclare à quiconque entend les paroles ...
31099,Apocalypse,22,19,et si quelqu'un retranche quelque chose des pa...,[et si quelqu'un retranche quelque chose des p...
31100,Apocalypse,22,20,"¶ celui qui atteste ces choses dit: oui, je vi...","[¶ celui qui atteste ces choses dit: oui, je v..."


In [7]:
data.columns

Index(['Book Name', 'Chapter', 'Verse', 'Text', 'text_chunks'], dtype='object')

In [8]:
from qdrant_client import models, QdrantClient
from langchain.embeddings import HuggingFaceEmbeddings
from sentence_transformers import util
import numpy as np

# Initialiser le client Qdrant
client = QdrantClient(":memory:")  # Stockage en mémoire, remplacez par la persistance si nécessaire

# Utiliser le modèle "BAAI/bge-m3" pour les embeddings
encoder = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

# Fonction pour stocker les vecteurs dans Qdrant
def vector_store1(data, encoder):
    # Créer une collection Qdrant avec la distance COSINE et la taille des vecteurs
    client.create_collection(
        collection_name="biblical_verses",
        vectors_config=models.VectorParams(
            distance=models.Distance.COSINE,
            size=encoder.model.get_sentence_embedding_dimension()  # Taille des embeddings
        ),
    )

    points = []
    point_id = 1
    for _, row in data.iterrows():
        # Encoder chaque chunk de texte
        for chunk in row['text_chunks']:  # Encodage de chaque partie du verset
            vector = encoder.embed_query(chunk)
            payload = {
                'Book Name': row['Book Name'],  # Nom du livre
                'Chapter': row['Chapter'],  # Chapitre
                'Verse': row['Verse'],  # Verset
                'Text': row['Text']  # Texte original complet
            }
            points.append(
                models.PointStruct(
                    id=point_id,  # ID unique pour chaque point
                    vector=vector,  # Embedding du chunk
                    payload=payload  # Metadonnées associées
                )
            )
            point_id += 1  # Incrémenter l'ID pour chaque point

    # Upload des points dans la collection
    client.upload_points(
        collection_name="biblical_verses",
        points=points,
    )

# Fonction pour rechercher des points similaires avec une question donnée
def search_similarity(question):
    hits = client.search(
        collection_name="biblical_verses",
        query_vector=encoder.embed_query(question),  # Encodage de la question
        limit=3,  # Limite des résultats à 3, ajustez si nécessaire
    )
    return hits

# Fonction pour récupérer les résultats sous forme de versets
def retriever(question):
    hits = search_similarity(question)
    responses = [hit.payload for hit in hits]  # Retourner les metadonnées associées aux versets trouvés
    return responses

# Fonction pour calculer la similarité cosinus entre les questions et les réponses
def cosine_similarity(questions, answers):
    similarities = []
    for question, answer in zip(questions, answers):
        results = search_similarity(question)
        for result in results:
            similarity = util.cos_sim(encoder.embed_query(result.payload['Text']), encoder.embed_query(answer))
            similarities.append(similarity)
    return np.mean(similarities)

# Fonction pour calculer la distance euclidienne entre deux embeddings
def euclidean_distance(embedding1, embedding2):
    embedding1 = np.array(embedding1)
    embedding2 = np.array(embedding2)
    distance = np.linalg.norm(embedding1 - embedding2)
    return distance


<ipython-input-8-27a59fc80184>:10: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  encoder = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [9]:
from sentence_transformers import SentenceTransformer

# Remplacez l'encodeur HuggingFaceEmbeddings par SentenceTransformer
encoder = SentenceTransformer("BAAI/bge-m3")

def vector_store(data, encoder):
    # Créer une collection dans Qdrant
    client.create_collection(
        collection_name="IA_Biblique",
        vectors_config=models.VectorParams(
            distance=models.Distance.COSINE,
            size=encoder.get_sentence_embedding_dimension()  # Taille des embeddings
        ),
    )

    points = []
    point_id = 1
    for _, row in data.iterrows():
        for chunk in row['text_chunks']:  
            vector = encoder.encode(chunk)  # Encodez chaque chunk
            payload = {
                'Book Name': row['Book Name'],
                'Chapter': row['Chapter'],
                'Verse': row['Verse'],
                'Text': row['Text']
            }
            points.append(
                models.PointStruct(
                    id=point_id,  # ID unique pour chaque point
                    vector=vector,
                    payload=payload
                )
            )
            point_id += 1  # Incrémenter l'ID pour chaque point

    # Téléverser les points dans la collection
    client.upload_points(
        collection_name="IA_Biblique",
        points=points,
    )


In [10]:
vector_store(data, encoder)

In [11]:
# Encoder la question
question = "que dit la bible de l'amour ?"
question_embedding = encoder.encode(question).tolist()  # Encode la question

# Effectuer la recherche dans Qdrant
search_result = client.search(
    collection_name="IA_Biblique",
    query_vector=question_embedding,
    limit=5  # Le nombre de résultats que tu souhaites obtenir
)

# Afficher les résultats
for result in search_result:
    print(result)


id=30611 version=0 score=0.5861722441924495 payload={'Book Name': '1 Jean', 'Chapter': 4, 'Verse': 7, 'Text': "¶ bien-aimés, aimons nous les uns les autres; car l'amour est de dieu, et quiconque aime est né de dieu et connaît dieu."} vector=None shard_key=None order_value=None
id=30612 version=0 score=0.5650973480999054 payload={'Book Name': '1 Jean', 'Chapter': 4, 'Verse': 8, 'Text': "celui qui n'aime pas n'a pas connu dieu, car dieu est amour."} vector=None shard_key=None order_value=None
id=30652 version=0 score=0.562886983216701 payload={'Book Name': '2 Jean', 'Chapter': 1, 'Verse': 6, 'Text': "et l'amour consiste à marcher selon ses commandements. c'est là le commandement dans lequel vous devez marcher, comme vous l'avez appris dès le commencement."} vector=None shard_key=None order_value=None
id=30247 version=0 score=0.5613264436745387 payload={'Book Name': 'Hébreux', 'Chapter': 13, 'Verse': 5, 'Text': "ne vous livrez pas à l'amour de l'argent; contentez- vous de ce que vous avez

In [12]:
# Encoder la question de l'utilisateur
question = "Combien de jour Dieu a t-il prit pour créer la terre?"
question_embedding = encoder.encode(question).tolist()  # Encode la question

# Rechercher les passages les plus proches dans l'index
search_result = client.search(
    collection_name="IA_Biblique",
    query_vector=question_embedding,
    limit=5  # Retourner les 5 versets les plus proches par exemple
)

# Extraire les passages récupérés
retrieved_passages = [result.payload['Text'] for result in search_result]

# Afficher les passages récupérés
for passage in retrieved_passages:
    print(passage)


qui l'a chargé de gouverner la terre? qui a confié l'univers à ses soins?
car, encore sept jours, et je ferai pleuvoir sur la terre quarante jours et quarante nuits, et j'exterminerai de la face de la terre tous les êtres que j'ai faits.
la pluie tomba sur la terre quarante jours et quarante nuits.
les eaux furent grosses sur la terre pendant cent cinquante jours.
pharaon dit à jacob: quel est le nombre de jours des années de ta vie?


In [13]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Charger le modèle de génération (GPT-2 ou un autre modèle adapté)
tokenizer_gen = GPT2Tokenizer.from_pretrained("gpt2")
model_gen = GPT2LMHeadModel.from_pretrained("gpt2")

# Préparer le contexte en combinant les passages récupérés
context = " ".join(retrieved_passages)

# Créer l'entrée pour le modèle de génération avec la question et le contexte
input_text = f"Context: {context} \nQuestion: {question} \nAnswer:"
input_ids = tokenizer_gen.encode(input_text, return_tensors='pt')

# Générer la réponse
outputs = model_gen.generate(input_ids, max_length=200, num_return_sequences=1)

# Décoder la réponse
generated_answer = tokenizer_gen.decode(outputs[0], skip_special_tokens=True)

# Afficher la réponse
print(generated_answer)


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Context: qui l'a chargé de gouverner la terre? qui a confié l'univers à ses soins? car, encore sept jours, et je ferai pleuvoir sur la terre quarante jours et quarante nuits, et j'exterminerai de la face de la terre tous les êtres que j'ai faits. la pluie tomba sur la terre quarante jours et quarante nuits. les eaux furent grosses sur la terre pendant cent cinquante jours. pharaon dit à jacob: quel est le nombre de jours des années de ta vie? 
Question: Combien de jour Dieu a t-il prit pour créer la terre? 
Answer: Je suis j'ai fait de


In [14]:
pip install Flask


In [47]:
!pip install Flask flask-ngrok

In [48]:
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)  # Démarre ngrok lorsque l'application Flask démarre

@app.route('/search', methods=['POST'])
def search():
    question = request.json.get('question')
    # Remplacez ceci par votre logique de recherche
    results = [{"book": "Genèse", "chapter": 1, "verse": 1, "text": "Au commencement, Dieu créa le ciel et la terre."}]
    return jsonify({"results": results})

if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-15:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connection.py", line 199, in _new_conn
    sock = connection.create_connection(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 789, in urlopen
    response = self._make_request(
  File "/usr/local/lib/python3.10/dist-packages/urll